In [7]:
search_for = 95
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022978782653808594
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 107435141
type: [1, 1, 1, 1, 95] 95
cases of this type: 81450625
10000 10.444140811396611
20000 11.996042314590891
30000 20.83648984852359
40000 46.055638256850386
50000 39.507038251297494
60000 41.829553881070254
70000 43.77988584403534
80000 50.20854739476782
90000 53.943933173810365
100000 52.272825904397564
110000 57.04299490766132
120000 64.63618321414948
130000 64.71690828499415
140000 62.59382625993816
150000 62.72665071494872
160000 62.121187630055964
170000 63.0698029470908
180000 65.74033040319804
190000 62.23836258722099
200000 75.46438767465892
210000 65.64268614603756
220000 84.04211287037725
230000 85.68183741208773
240000 72.82246798973678
250000 69.43735054749497
260000 64.8707764906795
270000 73.58032850380523
280000 60.814576426243875
290000 61.54035362030254
300000 66.17479255942393
310000 63.02608963147961
3

3100000 57.03768544626405
3110000 54.73954027654043
3120000 53.40110181224431
3130000 55.06915732935986
3140000 54.07507278915381
3150000 51.892428343335055
3160000 54.39181280451703
3170000 55.30944514162942
3180000 53.13246490734393
3190000 58.017336299145526
3200000 58.67020622267136
3210000 58.793228155476754
3220000 55.84981788786643
3230000 59.20221269346363
3240000 58.64660965389429
3250000 57.37289026730333
3260000 55.585832986625775
3270000 56.97023086512241
3280000 55.435357042362924
3290000 53.51973362247589
3300000 55.7932740527295
3310000 57.61830722203129
3320000 57.98767242754614
3330000 55.83588420725373
3340000 55.20442201780302
3350000 55.790805377749386
3360000 56.510778169744405
3370000 60.41237540907741
3380000 42.74056492098961
3390000 21.91284712438385
3400000 21.898403155518988
3410000 21.96124311449754
3420000 22.44573384492797
3430000 67.22714824446872
3440000 68.69651648022267
3450000 63.86758090440009
3460000 17.512434241028856
3470000 17.49942219044171
3480

6230000 52.45175623994669
6240000 56.400967049804365
6250000 56.453554184195646
6260000 58.07041822380065
6270000 53.2073618415531
6280000 54.145904083791876
6290000 54.878002466323736
6300000 58.6527685783298
6310000 52.382762098351385
6320000 52.375688978051365
6330000 54.03645896117453
6340000 52.395304073259915
6350000 50.69335106555329
6360000 53.9138817591715
6370000 54.55705156824279
6380000 50.217326182996636
6390000 46.89859231765818
6400000 49.418508229370815
6410000 51.75863356419372
6420000 8.6043530842137
6430000 8.533070928993935
6440000 8.525199693078875
6450000 8.590409423749493
6460000 55.66680437272476
6470000 58.3175439065786
6480000 44.49270856452963
6490000 48.500999085508795
6500000 52.0509937437747
6510000 54.71881577242689
6520000 51.530397860442775
6530000 51.60209228988954
6540000 50.99446787159676
6550000 53.05187098156781
6560000 52.45054188366007
6570000 54.03439624799794
6580000 55.13880479951183
6590000 54.66635293232101
6600000 52.29885595270217
6610000 

9360000 24.469212127131716
9370000 65.33835356448373
9380000 59.40083742198658
9390000 54.85672054944056
9400000 52.74927194603274
9410000 56.202169825420064
9420000 60.506883515407054
9430000 60.69753262914149
9440000 60.015045076838135
9450000 60.13224862755283
9460000 55.196561707560576
9470000 60.25359537141528
9480000 57.30351146015514
9490000 52.32106310095218
9500000 56.17288386764723
9510000 62.946502430867966
9520000 28.09621914722867
9530000 17.12570238740182
9540000 18.58523943528675
9550000 17.11830592628036
9560000 28.672832496841927
9570000 62.84735568865184
9580000 51.943612296396104
9590000 51.98017478000956
9600000 59.18054765856989
9610000 58.786101696927496
9620000 54.74400023997833
9630000 57.91738111660498
9640000 55.34515643998514
9650000 54.855318206110596
9660000 55.927829417367384
9670000 58.418685406384185
9680000 53.412603408445335
9690000 56.13459406518907
9700000 57.68405568590678
9710000 55.98339794562161
9720000 53.768255323000744
9730000 53.2745433057127

12400000 47.104133653463194
12410000 53.294392593900135
12420000 8.213424073080645
12430000 8.203586072405988
12440000 8.186752799636642
12450000 8.194375147684713
12460000 34.37522326038112
12470000 54.44077484188613
12480000 45.835136171754996
12490000 44.4107915113174
12500000 48.502987307062206
12510000 46.2521350970601
12520000 45.66151789246941
12530000 45.85011330313595
12540000 45.18674626820114
12550000 45.41301431667546
12560000 46.80518766151406
12570000 48.97310632309113
12580000 50.12604651015954
12590000 53.46889316391277
12600000 54.48883973676913
12610000 48.71685486077263
12620000 49.24882288587081
12630000 49.23500301884892
12640000 48.38946159455479
12650000 50.36498014095883
12660000 50.677318892065756
12670000 51.72032334909937
12680000 53.984918369527826
12690000 52.40965438400594
12700000 54.64275268892768
12710000 52.515382319736226
12720000 20.119007611518278
12730000 19.786868261346356
12740000 20.226245545262795
12750000 19.94587522053204
12760000 53.67135451

...

96200000 2.657733170934439
96210000 1.9906478922182522
96220000 1.6457084472588697
96230000 1.5006791658861836
96240000 2.0542881159925392
96250000 2.6707642837012675
96260000 3.217066695194642
96270000 2.0846515364108815
96280000 1.1067336462316446
96290000 2.144571420230627
96300000 2.8821951693338552
96310000 2.796733504436056
96320000 1.604613326068183
96330000 1.7523235750699175
96340000 3.0535987917679415
96350000 2.385777607394914
96360000 1.0609608338083294
96370000 0.8854187019140589
96380000 1.648125734360695
96390000 1.7272984484129479
96400000 0.8870105095347697
96410000 0.6158153403517008
96420000 0.8700215266822908
96430000 0.942130285157793
96440000 0.889561595387737
96450000 0.8602097928951251
96460000 0.6662344446088869
96470000 0.9254358042765259
96480000 0.8653389966158469
96490000 0.8679155357238187
96500000 0.8288110676896174
96510000 0.8539687257024314
96520000 0.9286407626650134
96530000 1.1865996691793599
96540000 1.2516777288518217
96550000 0.8993892988815573
9

99130000 1.2838803950043651
99140000 1.477957733090957
99150000 1.5356123724960686
99160000 1.2642344706463085
99170000 1.2356011317932605
99180000 1.1337450128570663
99190000 1.5401549046501186
99200000 0.96429345538342
99210000 1.2194307847081622
99220000 0.985117723891616
99230000 1.2251054680151343
99240000 1.0068732691941988
99250000 1.1121727827941512
99260000 1.7323584182841179
99270000 1.2557949336460763
99280000 0.898802513819966
99290000 1.7161053268455664
99300000 1.8639163634968399
99310000 1.3555590566203528
99320000 1.8780404911035498
99330000 1.4960264870690372
99340000 1.0764346111604903
99350000 1.2100460128457546
99360000 1.487846417492297
99370000 1.2143913699817723
99380000 1.1042817937804856
99390000 1.5718146177936065
99400000 1.5095172324960762
99410000 1.0973050229674974
99420000 1.1968055644304023
99430000 1.6358895933560067
99440000 1.2134754648326636
99450000 1.422468769917349
99460000 1.37830457800691
99470000 1.592714869346281
99480000 1.6165426244768368
99

102010000 0.9802347586133148
102020000 0.5686378930622339
best so far: 0
type: [1, 1, 1, 95, 1] 95
cases of this type: 857375
102030000 0.7431602871810264
102040000 0.24590186040988896
102050000 0.2831076613273025
102060000 0.2691254837557806
102070000 0.3428339339626961
102080000 0.2539432956138651
102090000 0.3152002798023224
102100000 0.33967556318471165
102110000 0.32546715233389534
102120000 0.355063765152514
102130000 0.3004177084836033
102140000 0.3370062958098054
102150000 0.2393411790333059
102160000 0.19482058958331083
102170000 0.30406949377600356
102180000 0.3377502421285841
102190000 0.3328728997878697
102200000 0.23524529732783636
102210000 0.22834879906211958
102220000 0.3358027409568032
102230000 0.33109662705248594
102240000 0.325093564860887
102250000 0.2845210978654623
102260000 0.2843075012346904
102270000 0.23972242577974
102280000 0.3367755394107236
102290000 0.3359288832791448
102300000 0.3360957577660481
102310000 0.32949579178981653
102320000 0.3106848599539399

104770000 0.1715872052115997
104780000 0.17233008306094008
104790000 0.1971027297518717
104800000 0.20040813632865745
104810000 0.18726086344108978
104820000 0.14808271063943043
104830000 0.19347023535825808
104840000 0.19121065102541449
104850000 0.1940003385589123
104860000 0.19173692304564846
104870000 0.1919103488804698
104880000 0.19093390640282631
104890000 0.1870636936440799
104900000 0.18449407954261698
104910000 0.18502138654812839
104920000 0.1563142180187702
104930000 0.15118522831839323
104940000 0.18768208205509848
104950000 0.4202577675218449
104960000 0.573912473632435
104970000 0.4699039003153509
104980000 0.5775204487555226
104990000 0.5606747556527655
105000000 0.4101369224107133
105010000 0.4190089328419831
105020000 0.30443452605676646
105030000 0.3764917798965772
105040000 0.3992601039532357
105050000 0.49835732202265665
105060000 0.5319155056198968
105070000 0.41842797208236326
105080000 0.5468414514473875
105090000 0.49914117903929284
105100000 0.5400857829295265

107300000 0.011450565303683281
107310000 0.010916394229763085
107320000 0.0170906618879305
107330000 0.013425527821977935
107340000 0.013280708729737334
107350000 0.009245165177557205
107360000 0.005568482887744903
107370000 0.004814015948832035
107380000 0.004433295682814386
107390000 0.006386837569018205
107400000 0.0053487994088596775
107410000 0.002935637877245744
107420000 0.0019789639786283175
best so far: 0
type: [5, 1, 1, 19, 1] 95
cases of this type: 6859
107430000 0.0005184844528900252
best so far: 0
type: [5, 1, 19, 1, 1] 95
cases of this type: 361
best so far: 0
type: [5, 19, 1, 1, 1] 95
cases of this type: 19
best so far: 0
type: [19, 1, 1, 1, 5] 95
cases of this type: 625
best so far: 0
type: [19, 1, 1, 5, 1] 95
cases of this type: 125
best so far: 0
type: [19, 1, 5, 1, 1] 95
cases of this type: 25
best so far: 0
type: [19, 5, 1, 1, 1] 95
cases of this type: 5
best so far: 0
type: [95, 1, 1, 1, 1] 95
cases of this type: 1
best so far: 0
all cases: 107435141
best: 0
